<a href="https://colab.research.google.com/github/MadChapa/Deep_Learning/blob/main/OptimizeCNNmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
import keras_tuner
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        13872     
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 48)                1115184   
                                                                 
 dense_1 (Dense)             (None, 10)                490       
                                                                 
Total params: 1,130,378
Trainable params: 1,130,378
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3146 - accuracy: 0.8839 - val_loss: 0.3403 - val_accuracy: 0.8725
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2932 - accuracy: 0.8919 - val_loss: 0.3699 - val_accuracy: 0.8743
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2762 - accuracy: 0.8985 - val_loss: 0.4206 - val_accuracy: 0.8720
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2621 - accuracy: 0.9044 - val_loss: 0.3999 - val_accuracy: 0.8827
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2526 - accuracy: 0.9071 - val_loss: 0.3947 - val_accuracy: 0.8762
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2292 - accuracy: 0.9146 - val_loss: 0.4814 - val_accuracy: 0.8720
Epoch 10/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2266 - accuracy: 0.9189 - val_loss: 0.4252 - val_accuracy